In [1]:
from torch.utils.data import DataLoader

from tqdm import tqdm

import torchvision.transforms as transforms
import torchvision

import torch.nn.functional as F
import torch.optim as optim
import torch.nn as nn 
import torch

import matplotlib.pyplot as plt
import numpy as np

BATCH_SIZE = 32
LEARNING_RATE = 1e-3
EPOCH = 50

/Users/bahk_insung/miniconda3/lib/python3.10/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: dlopen(/Users/bahk_insung/miniconda3/lib/python3.10/site-packages/torchvision/image.so, 0x0006): Symbol not found: __ZN2at4_ops19empty_memory_format4callEN3c108ArrayRefIxEENS2_8optionalINS2_10ScalarTypeEEENS5_INS2_6LayoutEEENS5_INS2_6DeviceEEENS5_IbEENS5_INS2_12MemoryFormatEEE
  Referenced from: <F0D48035-EF9E-3141-9F63-566920E60D7C> /Users/bahk_insung/miniconda3/lib/python3.10/site-packages/torchvision/image.so
  Expected in:     <11CA4EBF-5A84-3662-A35A-A039CEE97F5E> /Users/bahk_insung/miniconda3/lib/python3.10/site-packages/torch/lib/libtorch_cpu.dylib
  warn(f"Failed to load image Python extension: {e}")


In [2]:
device = torch.device('mps')

# Dataset

In [3]:
transforms = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

trainset   = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transforms)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=32, shuffle=True)

testset    = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transforms)
testloader = torch.utils.data.DataLoader(testset, batch_size=32, shuffle=True)

Files already downloaded and verified
Files already downloaded and verified


# Modeling

In [4]:
class ResidualBlock(nn.Module):
    def __init__(self, in_channels, out_channels, strides=1):
        super(ResidualBlock, self).__init__()
        self.strides = strides
        self.in_channels = in_channels
        self.out_channels = out_channels
        self.conv_block = nn.Sequential(
            nn.Conv2d(self.in_channels, self.out_channels, kernel_size=3, stride=strides, padding=1, bias=False),
            nn.BatchNorm2d(self.out_channels),
            nn.ReLU(),

            nn.Conv2d(self.out_channels, self.out_channels, kernel_size=3, stride=1, padding=1, bias=False),
            nn.BatchNorm2d(self.out_channels)
        )

        if self.strides != 1 or self.in_channels != self.out_channels:
            self.downsample = nn.Sequential(
                nn.Conv2d(self.in_channels, self.out_channels, kernel_size=1, stride=strides, bias=False),
                nn.BatchNorm2d(self.out_channels)
            )

    def forward(self, x):
        out = self.conv_block(x)
        if self.strides != self.in_channels != self.out_channels:
            x = self.downsample(x)

        out = F.relu(x + out)
        return out

In [5]:
class ResNet(nn.Module):
    def __init__(self, num_blocks, num_classes=10):
        super(ResNet, self).__init__()
        self.in_channels = 64
        self.base = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1, bias=False),
            nn.BatchNorm2d(64),
            nn.ReLU()
        )

        self.layer1 = self._make_layer(64,  num_blocks[0], strides=1)
        self.layer2 = self._make_layer(128, num_blocks[1], strides=2)
        self.layer3 = self._make_layer(256, num_blocks[2], strides=2)
        self.layer4 = self._make_layer(512, num_blocks[3], strides=2)
        self.gap    = nn.AvgPool2d(4)
        self.fc     = nn.Linear(512, num_classes)

    def _make_layer(self, out_channels, num_blocks, strides):
        strides = [strides] + [1] * (num_blocks - 1)
        layers = []

        for stride in strides:
            block = ResidualBlock(self.in_channels, out_channels, stride)
            layers.append(block)
            self.in_channels = out_channels

        return nn.Sequential(*layers)

    def forward(self, x):
        out = self.base(x)
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = self.gap(out)
        out = out.view(out.size(0), -1)
        out = self.fc(out)
        return out

def modeltype(model):
    if model == 'resnet18':
        return ResNet([2, 2, 2, 2])

    elif model == 'resnet34':
        return ResNet([3, 4, 6, 3])

In [6]:
resnet = modeltype('resnet18').to(device)
print(resnet)

critetion = nn.CrossEntropyLoss()
optimizer = optim.Adam(resnet.parameters(), lr=LEARNING_RATE)

ResNet(
  (base): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
  )
  (layer1): Sequential(
    (0): ResidualBlock(
      (conv_block): Sequential(
        (0): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): ReLU()
        (3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
    )
    (1): ResidualBlock(
      (conv_block): Sequential(
        (0): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): ReLU()
        (3): Conv2d(64, 64, kerne

In [7]:
PATH = './models/cifar_resnet.pth'

In [8]:
loss_ = list()
n = len(trainloader)

for epoch in range(EPOCH):
    running_loss = 0.0
    for data in trainloader:
        inputs, labels = data[0].to(device), data[1].to(device)
        optimizer.zero_grad()
        outputs = resnet(inputs)
        loss = critetion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    loss_.append(running_loss / n)
    print("[%d] loss : %.3f" % (epoch + 1, running_loss / len(trainloader)))

[1] loss : 1.300
[2] loss : 0.797
[3] loss : 0.602
[4] loss : 0.466
[5] loss : 0.363
[6] loss : 0.269
[7] loss : 0.189
[8] loss : 0.139
[9] loss : 0.101
[10] loss : 0.085
[11] loss : 0.070
[12] loss : 0.058
[13] loss : 0.055
[14] loss : 0.050
[15] loss : 0.042
[16] loss : 0.040
[17] loss : 0.039
[18] loss : 0.036
[19] loss : 0.033
[20] loss : 0.030
[21] loss : 0.029
[22] loss : 0.029
[23] loss : 0.025
[24] loss : 0.027
[25] loss : 0.021
[26] loss : 0.023
[27] loss : 0.021
[28] loss : 0.021
[29] loss : 0.019
[30] loss : 0.018
[31] loss : 0.018
[32] loss : 0.019
[33] loss : 0.015
[34] loss : 0.018
[35] loss : 0.014
[36] loss : 0.016
[37] loss : 0.016
[38] loss : 0.012
[39] loss : 0.017
[40] loss : 0.012
[41] loss : 0.012
[42] loss : 0.014
[43] loss : 0.013
